## Combine everything from Ergast DB

In [ ]:
import pandas as pd

# Load all CSV files
results = pd.read_csv('./f1db_csv2024-2/results.csv')
races = pd.read_csv('./f1db_csv2024-2/races.csv')
drivers = pd.read_csv('./f1db_csv2024-2/drivers.csv')
constructors = pd.read_csv('./f1db_csv2024-2/constructors.csv')
laptimes = pd.read_csv('./f1db_csv2024-2/lap_times.csv')
qualifying = pd.read_csv('./f1db_csv2024-2/qualifying.csv')
pitstops = pd.read_csv('./f1db_csv2024-2/pit_stops.csv')
driver_standings = pd.read_csv('./f1db_csv2024-2/driver_standings.csv')
constructor_standings = pd.read_csv('./f1db_csv2024-2/constructor_standings.csv')

pitstops = pitstops.drop(columns=['time'])
driver_standings = driver_standings.drop(columns=['position', 'positionText'])
constructor_standings = constructor_standings.drop(columns=['position', 'positionText'])


# Merge results with races
merged_df = pd.merge(results, races, on='raceId', how='left')

# Merge with drivers
merged_df = pd.merge(merged_df, drivers, on='driverId', how='left')


merged_df = pd.merge(merged_df, constructors, on='constructorId', how='left')

# Merge with laptimes
merged_df = pd.merge(merged_df, laptimes, on=['raceId', 'driverId'], how='left')

merged_df = pd.merge(merged_df, qualifying, on=['raceId', 'driverId'], how='left')

merged_df = pd.merge(merged_df, driver_standings, on=['raceId', 'driverId'], how='left')

print(merged_df.columns.tolist())
merged_df = merged_df.rename(columns={'constructorId_x': 'constructorId'})
merged_df = pd.merge(merged_df, constructor_standings, on=['raceId', 'constructorId'], how='left')

merged_df = pd.merge(merged_df, pitstops, on=['raceId', 'driverId', 'lap'], how='left')


merged_df = merged_df[merged_df['raceId'] >= 900]
columns_to_remove = ['position_x', 'positionText', 'time_x', 'url_x', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'url_y', 'url']
merged_df = merged_df.drop(columns=columns_to_remove)

merged_df = merged_df.rename(columns={'time': 'laptime', 'points_y':'driverseasonpoints', 'points':'constructorseasonpoints', 'position':'qualiposition', 'position_y':'raceposition', 'wins_x':'driverwins', 'wins_y':'constructorwins'})


merged_df['Driver_Season_Points'] = merged_df['driverseasonpoints'] - merged_df['points_x']

# Save the combined DataFrame to a new CSV file
merged_df.to_csv('./f1db_csv2024-2/combined_results.csv', index=False)


# The following cells must be done for each season separately 

## Adding more driver information to the dataset

In [ ]:
import pandas as pd

combinedresults = pd.read_csv('./f1db_csv2024-2/combined_results.csv')

drivers_dict = {
    1: {"Abbreviation": "VER", "YOB": 1997, "Races_before": 181, "Races_won": 54, "Podiums": 92},
    11: {"Abbreviation": "PER", "YOB": 1990, "Races_before": 253, "Races_won": 6, "Podiums": 33},
    44: {"Abbreviation": "HAM", "YOB": 1985, "Races_before": 328, "Races_won": 103, "Podiums": 196},
    14: {"Abbreviation": "ALO", "YOB": 1981, "Races_before": 373, "Races_won": 32, "Podiums": 104},
    16: {"Abbreviation": "LEC", "YOB": 1997, "Races_before": 122, "Races_won": 5, "Podiums": 28},
     4: {"Abbreviation": "NOR", "YOB": 1999, "Races_before": 100, "Races_won": 0, "Podiums": 12},
    55: {"Abbreviation": "SAI", "YOB": 1994, "Races_before": 182, "Races_won": 2, "Podiums": 17},
    63: {"Abbreviation": "RUS", "YOB": 1998, "Races_before": 100, "Races_won": 1, "Podiums": 10},
    81: {"Abbreviation": "PIA", "YOB": 2001, "Races_before": 18, "Races_won": 0, "Podiums": 2},
    18: {"Abbreviation": "STR", "YOB": 1998, "Races_before": 140, "Races_won": 0, "Podiums": 3},
    10: {"Abbreviation": "GAS", "YOB": 1996, "Races_before": 126, "Races_won": 1, "Podiums": 4},
    31: {"Abbreviation": "OCO", "YOB": 1996, "Races_before": 126, "Races_won": 1, "Podiums": 3},
    23: {"Abbreviation": "ALB", "YOB": 1996, "Races_before": 77, "Races_won": 0, "Podiums": 2},
     2: {"Abbreviation":"SAR","YOB" :2000,"Races_before" :22,"Races_won" :0,"Podiums" :0},
    22: {"Abbreviation":"TSU","YOB" :2000,"Races_before" :62,"Races_won" :0,"Podiums" :0},
    87: {"Abbreviation":"BEA","YOB" :2005,"Races_before" :0,"Races_won" :0,"Podiums" :0}, 
    43: {"Abbreviation":"COL","YOB" :2003,"Races_before" :0,"Races_won" :0,"Podiums" :0}, 
     24:{"Abbreviation":"ZHO","YOB" :1999,"Races_before" :40,"Races_won" :0,"Podiums" :0},
     30:{"Abbreviation":"LAW","YOB" :2002,"Races_before" :3,"Races_won" :0,"Podiums" :0},
     77:{"Abbreviation":"BOT","YOB" :1989,"Races_before" :218,"Races_won" :10,"Podiums" :67}, 
     3:{"Abbreviation":"RIC","YOB" :1989,"Races_before" :232,"Races_won" :8,"Podiums" :32}, 
     20:{"Abbreviation":"MAG","YOB" :1992,"Races_before" :160,"Races_won" :0,"Podiums" :1}
}


# Function to retrieve lap times for a given race
def get_race_laptimes(season, round):

    lap_times = []
    for driver in drivers_dict:
        laps = combinedresults[(combinedresults['year'] == season) & (combinedresults['round'] == round) & (combinedresults['number'] == driver)]
        

        if laps.empty:
            print("Driver "+str(driver)+" is not in round "+str(round))
            continue
        else:
            laps.loc[:,'YOB'] = drivers_dict[driver]['YOB']
            print("SUCCESS YOB")
            laps.loc[:,'Races_before'] = drivers_dict[driver]['Races_before']
            laps.loc[:,'Races_won'] = drivers_dict[driver]['Races_won']
            laps.loc[:,'Podiums'] = drivers_dict[driver]['Podiums']
            
            if(laps.loc[laps['number'] == driver]['points_x'].values[0]>=25):
                drivers_dict[driver]['Races_won']+=1
            
            if(laps.loc[laps['number'] == driver]['points_x'].values[0]>=15):
                drivers_dict[driver]['Podiums']+=1

            drivers_dict[driver]['Races_before']+=1


            lap_times.append(laps)

    
    print("Done with round "+str(round))
    return pd.concat(lap_times)

# Can loop through seasons and rounds if information is available for all seasons, or else just do it one season at a time
all_lap_times = []
for season in range(2024, 2025):

    for round in range(1, 25):  # Typically 22-24 rounds per season, adjust if necessary
        try:
            race_lap_times = get_race_laptimes(season, round)
            all_lap_times.append(race_lap_times)
        except Exception as e:
            print(f"Error retrieving data for season {season}, round {round}: {e}")


# Combine all lap times into a single DataFrame
all_lap_times_df = pd.concat(all_lap_times, ignore_index=True)

# Save to CSV
all_lap_times_df.to_csv('./f1db_csv2024-2/f1_lap_times_2024.csv', index=False)

print("Lap times successfully retrieved and saved to 'f1_lap_times_2024.csv'.")

## Adding safety car info

In [ ]:
safety_car_df = pd.read_csv('./f1db_csv2024/safety_cars.csv')
laptimes = pd.read_csv('./f1db_csv2024-2/f1_lap_times_2024.csv')




# Initialize a new column `isSafetyCar` with 0
laptimes['isSafetyCar'] = 0
laptimes['isSafetyCarPrev'] = 0

# Split the 'Race' column in safety_car_df into 'year' and 'name'
safety_car_df['year'] = safety_car_df['Race'].apply(lambda x: int(x.split()[0]))
safety_car_df['name'] = safety_car_df['Race'].apply(lambda x: ' '.join(x.split()[1:]))

safety_car_df = safety_car_df[safety_car_df['year'] == 2024]

# Iterate through each row in the safety car dataset
for _, row in safety_car_df.iterrows():
    race_year = row['year']
    race_name = row['name']
    deployed_lap = row['Deployed']
    retreated_lap = row['Retreated']
    
    # Set `isSafetyCar` to 1 for laps between deployed and retreated in the same race
    laptimes.loc[
        (laptimes['year'] == race_year) & 
        (laptimes['name_x'] == race_name) & 
        (laptimes['lap'] >= deployed_lap) & 
        (laptimes['lap'] <= retreated_lap), 'isSafetyCar'
    ] = 1

    laptimes.loc[
        (laptimes['year'] == race_year) & 
        (laptimes['name_x'] == race_name) & 
        ((laptimes['lap'] - 1)>= deployed_lap) & 
        ((laptimes['lap'] - 1) <= retreated_lap), 'isSafetyCarPrev'
    ] = 1

# Save the updated combined DataFrame to a new CSV file
laptimes.to_csv('./f1db_csv2024-2/f1_lap_times_2024_safetycar.csv', index=False)


## Getting tyre compound information

In [ ]:
import fastf1 as ff1

# Enable caching for faster retrieval
# ff1.Cache.enable_cache('cache')  # Create a cache folder

# Initialize a list to store the data
data = []

# Loop through the seasons (2018 to 2023)
for year in range(2024, 2025):
    # Get the season schedule for the given year
    schedule = ff1.get_event_schedule(year)
    
    # Loop through all the races (rounds) in the season
    for round_num in schedule['RoundNumber']:
        try:
            # Load the session data for the race
            session = ff1.get_session(year, round_num, 'R')  # 'R' stands for race session
            session.load()  # Load the session data
            
            # Check if session.laps exists and is not empty
            if session.laps.empty:
                print(f"No lap data for year {year}, round {round_num}")
                continue
            
            # Loop through all laps in the session
            for _, lap in session.laps.iterrows():  # Use .iterrows() to iterate through DataFrame rows
                # Ensure 'LapNumber' and 'Compound' exist in the data
                if 'LapNumber' in lap and 'Compound' in lap:
                    driver_lap = {
                        'year': year,
                        'round': round_num,
                        'driver': lap['Driver'],
                        'drivernumber': lap['DriverNumber'],
                        'lap_number': lap['LapNumber'],
                        'tyre_compound': lap['Compound']
                    }
                    data.append(driver_lap)
                else:
                    print(f"Missing data for lap in year {year}, round {round_num}")

        except Exception as e:
            print(f"Failed to load session for year {year}, round {round_num}: {e}")

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('./f1db_csv2024-2/compounds_2024.csv', index=False)

print("Data fetching and CSV export complete!")

## Combining safety car and tyre compound

In [ ]:
compounds = pd.read_csv('./f1db_csv2024-2/compounds_2024.csv')
laptimes = pd.read_csv('./f1db_csv2024-2/f1_lap_times_2024_safetycar.csv')

compounds = compounds.rename(columns={'lap_number':'lap', 'driver':'code'})

merged_df = pd.merge(laptimes, compounds, on=['year', 'round', 'code', 'lap'], how='left')


# Save the updated combined DataFrame to a new CSV file
merged_df.to_csv('./f1db_csv2024-2/f1_lap_times_2024_raw_complete.csv', index=False)

## Adding tyre age info

In [ ]:
df = pd.read_csv('./f1db_csv2024-2/f1_lap_times_2024_raw_complete.csv')

# Sort the dataset by raceId, driverId, and lap number
df = df.sort_values(by=['raceId', 'driverId', 'lap'])

# Initialize a new column for tyre age
df['tyre_age'] = 0

# Loop through each driver and race to calculate tyre age
for driver, driver_data in df.groupby(['raceId', 'driverId']):
    tyre_age = 1  # Tyre age starts at 1 (since the first lap after a pit stop is lap 1 of tyre life)
    
    for index, row in driver_data.iterrows():
        # If a pit stop occurs (row['stop'] > 0), set the next lap's tyre age to 1
        if row['stop'] > 0:  # Pit stop detected
            # Set the tyre age for the current lap
            df.at[index, 'tyre_age'] = tyre_age

            tyre_age = 1  # Reset tyre age to 0 for the current lap

        else:
            # Set the tyre age for the current lap
            df.at[index, 'tyre_age'] = tyre_age
        
            # Increment the tyre age for the next lap
            tyre_age += 1
        
        

# Save the updated DataFrame with tyre age to a new CSV
df.to_csv('./f1db_csv2024-2/f1_lap_times_2024_with_tyre_age.csv', index=False)

## Adding encodings for drivers, teams, and pitstops

In [ ]:
df = pd.read_csv('./f1db_csv2024-2/f1_lap_times_2024_with_tyre_age.csv')

# One-hot encode the 'tyre_compound' column
df = pd.get_dummies(df, columns=['tyre_compound'], prefix='tyre', drop_first=False)


# List of driver codes and corresponding column names
drivers = [
    ('VET', 'isVET'), ('ZHO', 'isZHO'), ('VER', 'isVER'), ('TSU', 'isTSU'), ('STR', 'isSTR'), 
    ('MSC', 'isMSC'), ('SAR', 'isSAR'), ('RIC', 'isRIC'), ('SAI', 'isSAI'), ('RUS', 'isRUS'), 
    ('PIA', 'isPIA'), ('PER', 'isPER'), ('OCO', 'isOCO'), ('NOR', 'isNOR'), ('MAG', 'isMAG'), 
    ('LEC', 'isLEC'), ('LAW', 'isLAW'), ('LAT', 'isLAT'), ('HUL', 'isHUL'), ('HAM', 'isHAM'), 
    ('GAS', 'isGAS'), ('DEV', 'isDEV'), ('BOT', 'isBOT'), ('BEA', 'isBEA'), ('ALO', 'isALO'), 
    ('ALB', 'isALB')
]

teams = {
    'isRBR': ['red_bull'],
    'isFER': ['ferrari'],
    'isMER': ['mercedes'],
    'isALP': ['alpine', 'renault'],
    'isMCL': ['mclaren'],
    'isALF': ['alfa', 'sauber'],
    'isAST': ['aston_martin', 'racing_point', 'force_india'],
    'isHAA': ['haas'],
    'isATR': ['alphatauri', 'torro_rosso', 'rb'],
    'isWIL': ['williams']
}

# Initialize all driver-related columns to 0
for _, column_name in drivers:
    df[column_name] = 0

for column in teams.keys():
    df[column] = 0



df['isPitting'] = 0
# Loop through each row and update the respective column based on the driver code
for index, row in df.iterrows():

    if row['stop'] > 0:  # Pit stop detected
        # print("row: "+str(index))
        df.at[index, 'isPitting'] = 1
    
    driver_code = row['code']
    
    # Set the respective driver's column to 1 if there's a match
    for code, column_name in drivers:
        if driver_code == code:
            df.at[index, column_name] = 1


    constructor_ref = row['constructorRef'].lower()  # Make lowercase for case-insensitive matching
    
    for column, constructor_list in teams.items():
        if any(constructor in constructor_ref for constructor in constructor_list):
            df.at[index, column] = 1


# Save the updated DataFrame to a new CSV
df.to_csv('./f1db_csv2024-2/f1_lap_times_2024_encodings.csv', index=False)